# MMT Intro

In this notebook, we will introduce MMT and do some basic logic development.

MMT knowledge is stored in modules that we call **theories**.
A theory contains a list of symbol declarations.
You can directly enter MMT theories in notebook cells.
GLIF will then write them to a file and important them to MMT.
You can change the location with the `archive` command.

## Entering unicode

MMT uses a lot of unicode characters.
You can enter them by typing e.g. `\alpha` and then pressing the `<Tab>` key to get the letter α (only works in code cells).
Generally, this is inspired by how you would enter them in LaTeX.
[Here](https://github.com/jfschaefer/GLIFkernel/blob/main/glif_kernel/unicode-latex-map) is the list of the currently supported characters.

MMT uses unicode characters as delimiters (to avoid using up too many of the symbols you might want to use in your logic):
* End of module: ❚ (`\MD<TAB>`)
* End of declaration: ❙ (`\DD<TAB>`)
* End of declaration part: ❘ (`\OD<TAB>`)

## Syntax of propositional logic

Let us start with a theory that specifies the syntax of propositional logic.
Basically, we will introduce a type for propositions, `prop`.
Then we can describe the connectives as functions. For example, negation would be function that takes one proposition and returns a new one, i.e. `not : prop ⟶ prop`.

The arrow (`⟶`) can be entered with `\longrightarrow<Tab>`. Because that's very inefficient (and you will need lots of arrows), there is the shortcut `\raa<Tab>`.

Typically, an MMT theory has a **meta theory**, which provides "the building blocks" for your theory.
We will often use `ur:?LF`, the Edinburgh Logical Framework, which provides `⟶`, and `type`, but also lambdas and dependent types.

Here is the first theory:

In [1]:
theory proplog_syntax_first_attempt : ur:?LF =
    prop : type ❙
    not : prop ⟶ prop ❙
    and : prop ⟶ prop ❙
    or : prop ⟶ prop ❙
❚

Successfully imported proplog_syntax_first_attempt.mmt

We can make another theory that uses `proplog_syntax_first_attempt` to make some propositions.
In this case, we will use `proplog_syntax_first_attempt` as the meta theory:

In [2]:
theory test0 : ?proplog_syntax_first_attempt =
    // make a few propositions (linguistically inspired) ❙
    johnlaughs: prop ❙
    marylaughs: prop ❙
    // we can also define propositions (instead of declaring them): ❙
    john_doesnt_laugh: prop ❘ = not johnlaughs ❙
    // MMT knows that `not johnlaughs` is of type `prop`, so we can omit it ❙
    mary_doesnt_laugh = not marylaughs ❙
❚

Successfully imported test0.mmt

Writing logical expressions like that can become somewhat unreadable.
MMT allows us to introduce notations for symbols (e.g. `¬ X` instead of `not X`).
In the notation string, we use `1`, `2`, etc. for the placement of the argument.
That lets us define infix operators.
We can also specify precedences (higher precedences bind stronger).

Typically, when defining a logic, we try to have a minimal set of core connectives
(e.g. ¬ and ∧) and then say that the other connectives are just "syntactic sugar",
or defined in terms of the other connectives.
We can do the same in MMT by providing a definition for e.g. `or` using De Morgan's law.
We will say that A ∨ B is defined as ¬(¬A ∧ ¬B).
But instead of this syncategorematic definition, we will have to define the meaning of `or`
directly, which we can do with a lambda function:

    ∨ = λA.λB.¬(¬A∧¬B)

In MMT, we use the notation `[x] M` for λx.M.

In [3]:
theory proplog_syntax : ur:?LF =
    // We introduce `o` as a notation for `prop` ❙
    prop : type ❘ # o ❙
    not : o ⟶ o ❘ # ¬ 1 prec 100 ❙
    and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 prec 80 ❙
    or : o ⟶ o ⟶ o ❘ # 1 ∨ 2 prec 70 ❘ = [a, b] ¬(¬a ∧ ¬b) ❙
    impl : o ⟶ o ⟶ o ❘ # 1 ⇒ 2 prec 60 ❘ = [a, b] ¬a ∨ b ❙
❚

Successfully imported proplog_syntax.mmt

Now we have a theory for the syntax for propositional logic.
There is another logic that we will be interested in: **PLNQ** (Predicate Logic, No Quantifiers).
It's first-order logic without quantifiers.
For this, we need to add a new type (for what we will call individuals).

In MMT, we can develop theories in a modular way.
So the new theory will include everything from `proplog_syntax` and add the new type.
(Note: we could have developed `proplog_syntax` in a more modular way too).

In [4]:
theory plnq_syntax : ur:?LF =
    include ?proplog_syntax ❙    // include everything from proplog_syntax ❙
    individual : type ❘ # ι ❙
❚

Successfully imported plnq_syntax.mmt

And let's test one more time:

In [5]:
theory test1 : ?plnq_syntax =
    john : ι ❙
    mary : ι ❙
    run : ι ⟶ o ❙

    if_john_runs_then_mary_doesnt_run : o ❘
        = run john ⇒ ¬ run mary ❙
❚

Successfully imported test1.mmt

MMT type-checks everything we do.
So we get an error if we e.g. pretend the `run` is a proposition:

In [6]:
theory test2 : ?plnq_syntax =
    include ?test1 ❙  // import constants from above ❙

    test_prop = run john ∨ ¬ run ❙
❚

Errors invalid unit: http://mathhub.info/tmpGLIF/default?test2?test_prop?definition: Judgment |-- (run john)∨¬run :: o

## Theorems and proofs

Typically, we want to talk about what is true in a particular situation.
For this, we will introduce a new constant `ded` with the notation `⊢`:

In [7]:
theory proof : ur:?LF =
    include ?proplog_syntax ❙    // we only need `prop`, but we did not put it in a separate theory ❙

    ded : o ⟶ type ❘ # ⊢ 1 prec 5 ❙
❚

Successfully imported proof.mmt

The idea is that we can write `⊢ φ` to state that `φ` is true.
If you look at the "type" of `⊢`, we see that `⊢ φ` is a `type`.
This is the idea behind the **judgements as types** paradigm.
If you have not seen it before, this takes some getting used to.

One way to think about it is that `⊢ φ` is the *type of all proofs of `φ`*.
So if we say
```
    x : ⊢ φ
```
then we say that `x` is a proof of `φ`.
We can think of this as an **axiom**.

A **theorem** is like an axiom, except that we provide a proof term:
```
    x : ⊢ φ ❘ = "some term of type ⊢φ..."
```

To create proof terms, we need proof construction rules.
For example, if we have a proof of `φ` and a proof of `ψ`, then we want to be able to construct a proof of type `φ∧ψ`.
For this, we can create a function `andI` of type `⊢φ ⟶ ⊢ψ ⟶ ⊢φ∧ψ`,
which means that the function takes a proof of `φ` and a proof `ψ` and returns a proof of `φ∧ψ` – exactly as we wanted.
This is called "conjunction introduction" in the natural deduction calculus.
As this will have to work for all propositions `φ` and `ψ`, we will use another trick:
the Π operator (or dependent type operator).
In MMT it is written in curly braces:
```
    andI : {φ:o, ψ:o} ⊢φ ⟶ ⊢ψ ⟶ ⊢φ∧ψ
```
`andI` now takes 4 arguments: a proposition φ, a proposition ψ, a proof of φ, a proof of ψ, and then it returns a proof of φ∧ψ.

Let us now see the whole thing in practice:

In [8]:
theory test3 : ?proof =
    // Two propositions. ❙
    johnruns : o ❙
    maryruns : o ❙
    
    // Note: Just because `johnruns` is a proposition, this doesn't mean that it is actually true. ❙
    // There are lots of false propositions. However, we can use an axiom to state that it is true: ❙
    axiom1 : ⊢ johnruns ❙
    axiom2 : ⊢ ¬maryruns ❙

    // Here is our proof construction rule: ❙
    andI : {φ:o, ψ:o} ⊢φ ⟶ ⊢ψ ⟶ ⊢φ∧ψ ❙

    // And now we can make a first theorem: ❙
    theorem1 : ⊢johnruns ∧ ¬maryruns ❘
        = andI johnruns (¬maryruns) axiom1 axiom2 ❙
        //         φ        ψ         ⊢φ     ⊢ψ   ❙
❚

Successfully imported test3.mmt

Due to the way we described proof rules etc., proof checking boils down to type checking.
If we provide a wrong proof, the type checker will complain:

In [9]:
theory test4 : ?proof =
    include ?test3 ❙
    theorem2 : ⊢johnruns ∧ maryruns ❘
        = andI johnruns maryruns axiom1 axiom2 ❙
❚

Errors invalid unit: http://mathhub.info/tmpGLIF/default?test4?theorem2?definition: Judgment |-- andI johnruns maryruns axiom1 axiom2 :: ⊢johnruns∧maryruns